In [1]:
from pyspark.sql import SparkSession


In [5]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("NikAmooz") \
    .getOrCreate()

In [6]:
df=spark.range(100)
print(df.sample(0.06).collect())

[Row(id=31), Row(id=37), Row(id=39), Row(id=60), Row(id=68), Row(id=86)]


In [4]:

#Create RDD from parallelize    
dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd=spark.sparkContext.parallelize(dataList)
rdd.collect()

[('Java', 20000), ('Python', 100000), ('Scala', 3000)]

In [21]:
print("SparkContext version:\t", spark.version) # return SparkContext version


SparkContext version:	 3.1.2


In [18]:
from datetime import datetime
from pyspark.sql import Row

allTypes = spark.sparkContext.parallelize([Row(i=1, s="string", d=1.0, l=1,
            b=True, list=[1, 2, 3], dict={"s": 0}, row=Row(a=1),
            time=datetime(2014, 8, 1, 14, 1, 5))])
df = allTypes.toDF()
df.createOrReplaceTempView("allTypes")
spark.sql('select i+1, d+1, not b, list[1], dict["s"], time, row.a from allTypes where b and i > 0').collect()


[Row((i + CAST(1 AS BIGINT))=2, (d + CAST(1 AS DOUBLE))=2.0, (NOT b)=False, list[1]=2, dict[s]=0, time=datetime.datetime(2014, 8, 1, 14, 1, 5), a=1)]

In [10]:
df.rdd.map(lambda x: (x.i, x.s, x.d, x.l, x.b, x.time, x.row.a, x.list)).collect()


[(1,
  'string',
  1.0,
  1,
  True,
  datetime.datetime(2014, 8, 1, 14, 1, 5),
  1,
  [1, 2, 3])]

In [17]:
#first , create /data/csv folder 
#  - docker exec -it namenode bash
#  cd /data
#  hadoop fs -mkdir -p /data/csv
#  hadoop fs -put ./csv/* /data/csv

df=spark.read.csv("hdfs://namenode:8020/data/csv/Region.csv",header="true")
df.show(5)

+-----+--------+-----------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
| code|province|       city| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+-----+--------+-----------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|10000|   Seoul|      Seoul|37.566953|126.977977|                    607|               830|              48|         1.44|                   15.38|                5.8|             22739|
|10010|   Seoul| Gangnam-gu|37.518421|127.047222|                     33|                38|               0|         4.18|                   13.17|                4.3|              3088|
|10020|   Seoul|Gangdong-gu|37.530492|127.123837|           

In [14]:
spark.stop()